In [4]:
import db_builder.db_handler as dbh
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import calendar
import time
from torchvision import transforms
from tqdm import tqdm

In [7]:
db_params = {
    'user': 'postgres',
    'password': input('Please enter DB pw'),  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'bigblock', #tensionTerminator
}

In [77]:
toolcheck = dbh.DB_Conn(db_params)
toolcheck.connect()
filepath_with_trigger = toolcheck.get_filepath_with_trigger()
filepath_with_duoballs = toolcheck.get_filepath_with_duoballs()
toolcheck.disconnect()

Connected to PostgreSQL, DB: bigblock
2023-10-01 12:57:20,622 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-10-01 12:57:20,624 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-01 12:57:20,625 INFO sqlalchemy.engine.Engine select current_schema()
2023-10-01 12:57:20,626 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-01 12:57:20,627 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-10-01 12:57:20,627 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-01 12:57:20,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 12:57:20,629 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop JOIN tools_loop_association ON loop.id = tools_loop_association.loop_id 
WHERE loop.human_labeled = true AND tools_loop_association.tool_id = %(tool_id_1)s
2023-10-01 12:

In [78]:
#filepath_with_trigger

In [79]:
#filepath_with_duoballs

In [80]:
def trigger_crop(image):
    crop_box = (400, 450, 550, 550)
    cropped_image = transforms.functional.crop(image, *crop_box)
    return cropped_image

In [81]:
def videoToImageConverter(source_path: str, output_path: str):
    vidcap = cv2.VideoCapture(source_path)
    os.makedirs(output_path, exist_ok=True)
    count = 0

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    fps = 1

    while count < total_frames:
        success, image = vidcap.read()

        if count % int(fps) == 0:
            if not image is None and not image.size == 0:
                transform_test = transforms.ToPILImage()
                image = transform_test(image)
                image = trigger_crop(image)
                image = np.asarray(image)
                cv2.imwrite(f"{output_path}/{count}.png", image)  # save frame as PNG file

        count += 1

    vidcap.release()
    cv2.destroyAllWindows()

In [82]:
for x in tqdm(filepath_with_trigger):
    videoToImageConverter(x, './data/test_03/trigger')

100%|██████████| 74/74 [24:35<00:00, 19.94s/it]


In [83]:
for x in tqdm(filepath_with_duoballs):
    videoToImageConverter(x, './data/test_03/duoballs')

100%|██████████| 54/54 [11:55<00:00, 13.24s/it]
